In [1]:
import pandas as pd 
import numpy as np 

### [코스닥 기업 리스트] 엑셀파일에서 '회사명' 데이터 추출

In [2]:
KOSDAQ_df = pd.read_excel('코스닥기업리스트.xlsx')

print(KOSDAQ_df.shape)

queries_CMP = KOSDAQ_df['회사명'].tolist()
queries_CMP


(1388, 9)


['블리츠웨이',
 '석경에이티',
 '지놈앤컴퍼니',
 '프리시젼바이오',
 '알체라',
 '하이딥',
 '에프앤가이드',
 '티엘비',
 '인바이오',
 '엔젠바이오',
 'DB금융스팩8호',
 '퀀타매트릭스',
 '클리노믹스',
 '포인트모바일',
 '앱코',
 '엔에프씨',
 '제일전기공업',
 '하나기술',
 '티앤엘',
 '고바이오랩',
 '네패스아크',
 '소룩스',
 '위드텍',
 '센코',
 '바이브컴퍼니',
 '미코바이오메드',
 '피플바이오',
 '넥스틴',
 '원방테크',
 '비나텍',
 '씨엔알리서치',
 '박셀바이오',
 '비비씨',
 '교보10호기업인수목적',
 '핌스',
 '압타머사이언스',
 '이오플로우',
 '카카오게임즈',
 '피엔케이피부임상연구센타',
 '아이디피',
 '셀레믹스',
 '미투젠',
 '브랜드엑스코퍼레이션',
 '영림원소프트랩',
 '한국파마',
 '솔브레인',
 '이루다',
 '와이팜',
 '이엔드디',
 '엠투아이',
 '더네이쳐홀딩스',
 '제놀루션',
 '솔트룩스',
 '티에스아이',
 '에이프로',
 '소마젠',
 '신도기연',
 '위더스제약',
 '엠브레인',
 '젠큐릭스',
 'HB솔루션',
 '에스에이티이엔지',
 '에스씨엠생명과학',
 '휴럼',
 '이지바이오',
 '드림씨아이에스',
 '엔피디',
 '플레이디',
 '서울바이오시스',
 '제이앤티씨',
 '켄코아에어로스페이스',
 '레몬',
 '서남',
 '위세아이텍',
 '세림B&G',
 'CJ 바이오사이언스',
 '피피아이',
 'RF머트리얼즈',
 '아이비김영',
 '브릿지바이오',
 '다보링크',
 '메드팩토',
 '원텍',
 '신테카바이오',
 '제이엘케이',
 '태웅로직스',
 '윙스풋',
 '리메드',
 '비올',
 '코리아센터',
 '웨이버스',
 '노터스',
 '티움바이오',
 '씨에스베어링',
 '우양',
 '에스제이그룹',
 '제테마',
 '라파스',
 '아이티엠반도체',
 '일승',
 '미디어젠',
 '모코엠시스',
 '티

### [잡플래닛] 사이트 크롤링

In [17]:
#잡플래닛 수집 코드
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import datetime
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [4]:
login_url = 'https://www.jobplanet.co.kr/users/sign_in'

#email 본인 아이디, password 본인 패스워드 입력 단, 리뷰를 남겨서 전체 접근이 가능한 상태여야함
#현재 크롤링 코드에서는 아이디 비밀번호 입력 제외
email = 'dnrl52@naver.com'
password = 'son1403***'


LOGIN_INFO ={
    'user[email]' : email,
    'user[password]' : password,
    'commit' : '로그인'
}

### 크롤링 결과값 전처리

In [5]:
session = requests.session()

res = session.post(login_url, data = LOGIN_INFO, verify = False)

res.raise_for_status()
result = []

def clean_str(text):
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    text = text.replace('\r','. ')
    return text

def get_number(text):
  pattern =  "/([0-9]+)/"
  result = re.search(pattern, text)
  return result.group(1)

In [15]:
queries = queries_CMP

for n,query in enumerate(queries):
  
  url = 'https://www.jobplanet.co.kr/search/companies/'+str(query)
  #urls 만들기
  #bs로 해당 서치에 query와 똑같은 text의 주소 저장 => urls로 보내기
  while True:

    res = session.get(url)
    
    try:
      res.raise_for_status()  # 응답코드 200(정상)이 아니면 예외처리 및 프로그램 종료
      soup = BeautifulSoup(res.text, 'html.parser')
      break
    
    except: time.sleep(3)

  url_set = [i.a['href'] for i in soup.find_all('dt','us_titb_l3') if i.a.text in query]

# 각 기업별 리뷰

  res_set =[]
  urls =[]

  for i in url_set:
    urls.append('https://www.jobplanet.co.kr/'+i)

  for i in urls:
    res_set.append(session.get(i).url)

  for j in res_set:
    
  #url 은 보고싶은 기업의 리뷰 URL 이며 마지막은 ?page= 형태로 해야함, last_page는 해당 기업 리뷰의 마지막 페이지 입력
    last_page = 6
    time.sleep(3)
    for idx in range(1,last_page):
        url = j+'?page='+str(idx)
        res = session.get(url)
        res.raise_for_status()

        soup = BeautifulSoup(res.text, 'html.parser')

        count3 = 0
        count4 = 0
        count5 = 0

        try:
            for k in range(5):
                reviewer_info = []
                # 응답자 정보
                position = soup.select_one('.content_top_ty2 > span.txt1')[0 + count4].text
                status = soup.select_one('.content_top_ty2 > span.txt1')[1 + count4].text
                loc = soup.select_one('.content_top_ty2 > span.txt1')[2 + count4].text

                day = soup.select_one('.content_top_ty2 > span.txt1')[3+ count4].text

                # 점
                star_rating = soup.select_one('.us_star_m > div.star_score')[0+k]['style'][6:-1]

                # rating 5*5
                promotion = soup.select_one('.bl_score')[0 + count5]['style'][6:-1]
                welfare = soup.select_one('.bl_score')[1 + count5]['style'][6:-1]
                balance = soup.select_one('.bl_score')[2 + count5]['style'][6:-1]
                culture = soup.select_one('.bl_score')[3 + count5]['style'][6:-1]
                top = soup.select_one('.bl_score')[4 + count5]['style'][6:-1]
                # 중심 제목
                content = soup.select_one('h2.us_label')[0+k].text
                # 장단점 경영진 의견
                merit = soup.select_one('dl.tc_list > dd.df1 > span')[0 + count3].text
                disadvantages = soup.select_one('dl.tc_list > dd.df1 > span')[1 + count3].text
                df_tit = soup.select_one('dl.tc_list > dd.df1 > span')[2 + count3].text


                reviewer_info = [position, status, loc, day, star_rating, promotion, welfare, balance, culture, top,clean_str(content),clean_str(merit),clean_str(disadvantages), clean_str(df_tit),get_number(url),query]

                result.append(reviewer_info)
                reviewer_info=[]
                count3 += 3
                count4 += 4
                count5 += 5
                # print("pass :"+str(idx)+"-"+str(k))
        except :
            print("fail :" + str(idx))
            pass

    colname = ['직무','상황','지역','작성일','총점','승진 기회 및 가능성','복지 및 급여','업무와 삶의 균형','사내문화','경영진','총평','장점','단점','바라는점','잡플래닛 company Id','회사명']
 
    df = pd.DataFrame(result,columns=colname)

    # #200개의 회사 마다 checkpoint 생성
    # if n%200==0:
    #   print("###################SAVE CHPT####################")

    #   df.to_excel("jobplanet"+str(n)+".xlsx")

fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5
fail :1
fail :2
fail :3
fail :4
fail :5


In [21]:

# ESG_jobplanet5.xlsx 여기까지 작업함


# filename = 'ESG_jobplanet_'+datetime.datetime.now().strftime
# ('%Y-%m-%d %H %M')+'.xlsx'

# df.to_excel('filename+'.xlsx') 
df

# print(filename) #worktogether_2018-05-20 11 47.xls

,직무,상황,지역,작성일,총점,승진 기회 및 가능성,복지 및 급여,업무와 삶의 균형,사내문화,경영진,총평,장점,단점,바라는점,잡플래닛 company Id,회사명
